# Categorische variabelen

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Lees de gegevens in
data_bestand = '../data/data.csv'
data = pd.read_csv(data_bestand)

# Splits tussen doel van voorspellers
y = data.Price
X = data.drop(['Price'], axis=1)

# Verdeel de gegevens tussen training en test
X_train_full, X_test_full, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Wis de kolommen met missende waarden (simpelste oplossing)
kolommen_met_onbrekende_gegevens = [kol for kol in X_train_full.columns if X_train_full[kol].isnull().any()] 
X_train_full.drop(kolommen_met_onbrekende_gegevens, axis=1, inplace=True)
X_test_full.drop(kolommen_met_onbrekende_gegevens, axis=1, inplace=True)

# "Kardinaliteit" betekent het aantal unieke waarden in een kolom
# Selecteer categorische kolommen met relatief lage kardinaliteit (handig maar willekeurig)
lage_kardinaliteit_kolommen = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]

# Numerieke kolommen selecteren
Numerieke_kolommen = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
mijn_kolommen = lage_kardinaliteit_kolommen + Numerieke_kolommen
X_train = X_train_full[mijn_kolommen].copy()
X_test = X_test_full[mijn_kolommen].copy()

In [ ]:
X_train.head()

In [ ]:
# Haal lijst op met categorische variabelen
s = (X_train.dtypes == 'object')
object_kolommen = list(s[s].index)

print("Categorische variabelen:")
print(object_kolommen)

## kwaliteit van elke benadering te meten

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Functie voor het vergelijken van verschillende benaderingen
def score_dataset(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    return mean_absolute_error(y_test, preds)

### Benadering 1 (categorische variabelen verwijderen)

In [ ]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_test = X_test.select_dtypes(exclude=['object'])

print("MAE van Benadering 1 (Ordinale codering):")
print(score_dataset(drop_X_train, drop_X_test, y_train, y_test))

### Benadering 2 (Ordinale codering)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Kopie maken om te voorkomen dat originele data verandert
label_X_train = X_train.copy()
label_X_test = X_test.copy()

# Ordinale encoder toepassen op elke kolom met categorische data
ordinale_encoder = OrdinalEncoder()
label_X_train[object_kolommen] = ordinale_encoder.fit_transform(X_train[object_kolommen])
label_X_test[object_kolommen] = ordinale_encoder.transform(X_test[object_kolommen])

print("MAE van Benadering 2 (Ordinale codering):")
print(score_dataset(label_X_train, label_X_test, y_train, y_test))

### Benadering 3 (One-Hot-codering)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Pas one-hot encoder toe op elke kolom met categorische gegevens
OH_encoder = OneHotEncoder(handle_unknown='ignore')
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_kolommen]).toarray())  # Convert to dense array
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[object_kolommen]).toarray())        # Convert to dense array

# One-hot encoding verwijderd index; zet het terug
OH_cols_train.index = X_train.index
OH_cols_test.index = X_test.index

# Verwijder categorische kolommen (zal vervangen worden door one-hot encoding)
num_X_train = X_train.drop(object_kolommen, axis=1)
num_X_test = X_test.drop(object_kolommen, axis=1)

# Voeg one-hot gecodeerde kolommen toe aan numerieke features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

# Zorg ervoor dat alle kolommen een stringtype hebben
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_test.columns = OH_X_test.columns.astype(str)

print("MAE van Approach 3 (One-Hot Encoding):")
print(score_dataset(OH_X_train, OH_X_test, y_train, y_test))